# $H_4$ Ground State VQE

In [1]:
import tequila as tq
import numpy as np
import matplotlib.pyplot as plt
import cirq

/home/liamj/chemical-graph/tequila-beta/lib/python3.12/site-packages/tequila/simulators/simulator_api.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# Define the geometry
g = "h 0.0 0.0 0.0\nh 0.0 0.0 1.5\nh 0.0 0.0 3.0\nh 0.0 0.0 4.5"

# Get two molecules (one with JW, one with the new Parity tapering)
mol = tq.Molecule(backend="pyscf", geometry=g, basis_set="sto-3g", frozen_core=True)
mol2 = tq.Molecule(backend="pyscf", geometry=g, transformation='TaperedBinary', basis_set="sto-3g", frozen_core=True)

H = mol.make_hamiltonian()
H2 = mol2.make_hamiltonian()

U = mol.make_ansatz("spa", edges=[(0,1), (2,3)])
U2 = mol2.make_ansatz("spa", edges=[(0,1), (2,3)])

In [ ]:
# Define a variable - what is this actually doing?
var = (tq.Variable("R0") + 0.5) * np.pi

# Spatial orbital rotations (UR(i, j, theta) rotates spatial orbitals i and j by angle theta )
# Why do we need these???? - they are necessary to optimize the orbitals in the quantum case. They are unneccessary in the classical case, however
UR0 = mol.UR(0,1,var) + mol.UR(2,3, var)
U = U + UR0

UR0 = mol2.UR(0,1,var) + mol2.UR(2,3, var)
U2 = U2 + UR0

E = tq.ExpectationValue(U=U, H=H)
E2 = tq.ExpectationValue(U=U2, H=H2)

res = tq.minimize(E, silent=True)
res2 = tq.minimize(E2, silent=True)

print("difference JW and parity", abs(res.energy - res2.energy)*1000, "meh")

difference JW and parity 1.3322676295501878e-12 meh


In [4]:
res2.energy

-1.5561260779839963

In [5]:
res.energy

-1.5561260779839976

In [149]:
res_dict = dict(res.variables)
res2_dict = dict(res2.variables)

In [8]:
try:
    U.export_to(filename="h4_g1_jordan_wigner.pdf")
    U2.export_to(filename="h4_g1_parity_taper.pdf")
except Exception as E:
    print("no qpic and/or latex installed ... sorry")

tikz2preview h4_g1_jordan_wigner.tikz
pdflatex -interaction=batchmode h4_g1_jordan_wigner.tex
This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
tikz2preview h4_g1_parity_taper.tikz
pdflatex -interaction=batchmode h4_g1_parity_taper.tex
This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode


In [6]:
E_circuit = tq.compile(E, backend="cirq").get_expectationvalues()[0].U.circuit
E2_circuit = tq.compile(E2, backend="cirq").get_expectationvalues()[0].U.circuit

In [7]:
E_circuit

0: ───X───────────────────────────────────────────────────X───@───
                                                          │   │
1: ───────────────────────────────────────────────────────┼───X───
                                                          │
2: ───Y^(0.318309886183791*f((((0, 1), 'D', None),))_0)───@───@───
                                                              │
3: ───────────────────────────────────────────────────────────X───

4: ───X───────────────────────────────────────────────────X───@───
                                                          │   │
5: ───────────────────────────────────────────────────────┼───X───
                                                          │
6: ───Y^(0.318309886183791*f((((2, 3), 'D', None),))_1)───@───@───
                                                              │
7: ───────────────────────────────────────────────────────────X───

In [126]:
hamiltonian_keys = list(H.keys())
hamiltonian_values = list(H.values())

In [136]:
param_R0

-0.5000001078407643

In [135]:
param_R0 = res_dict["R0"]
param_1 = res_dict[list(res_dict.keys())[0]]
param_2 = res_dict[list(res_dict.keys())[1]]

simulator = cirq.Simulator()

parameterized_angles = []

for parameterized_angle in range(-300, 301, 20):

    new_ops = []

    for moment in E_circuit:
        for op in moment:

            if op.gate not in [cirq.CNOT, cirq.H, cirq.X, cirq.CZ]:
                if type(op.gate.exponent) != float:
                    if "R0" in str(op.gate.exponent.args[1]):
                        new_ops.append(cirq.YPowGate(exponent=float(op.gate.exponent.args[0])*param_R0, global_shift=op.gate.global_shift).on(op.qubits[0]))
                    elif "(0, 1)" in str(op.gate.exponent.args[1]):
                        new_ops.append(cirq.YPowGate(exponent=float(op.gate.exponent.args[0])*param_1, global_shift=op.gate.global_shift).on(op.qubits[0]))
                    else:
                        new_ops.append(cirq.YPowGate(exponent=float(op.gate.exponent.args[0])*param_2, global_shift=op.gate.global_shift).on(op.qubits[0]))
                    
                    new_ops.append(cirq.YPowGate(exponent=parameterized_angle * 0.01 * 0.31830988618379069, global_shift=-0.5).on(cirq.LineQubit(1)))
                else:
                    new_ops.append(op)

            else:
                new_ops.append(op)

    new_circuit_from_optimization = cirq.Circuit(new_ops)

    result = simulator.simulate(new_circuit_from_optimization)
    parameterized_angles.append(parameterized_angle * 0.01)